In [28]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.gridspec as gridspec
from matplotlib.patches import Patch
import os

In [29]:
save_path = '../assets'
if not os.path.exists(save_path): os.makedirs(save_path)

In [ ]:
plt.rcParams['font.family'] = 'sans-serif'
sns.set_style("white")

# Cores
COLOR_FRAUD = "#003366"   # Azul Escuro
COLOR_NORMAL = "#0085FF"  # Azul Claro
COLOR_TEXT = "#333333"
PALETTE_PRED = [COLOR_NORMAL, COLOR_FRAUD]
PALETTE_ROLES = {'Honest': '#F26522', 'Mule': '#0085FF', 'Boss': '#0D1B5E', 'Unknown': 'gray'}


# --- 2. CARREGAR DADOS ---
try:
    df = pd.read_csv('../data/01_raw/resultados_finais_modelagem.csv', index_col=0)
    if 'predicao_classe' in df.columns:
        df = df.rename(columns={'predicao_classe': 'is_fraud_pred'})
    
    # Raw para contagem total
    df_raw = pd.read_csv('../data/01_raw/synthetic_dataset.csv')
    total_transacoes = len(df_raw)
    del df_raw
except FileNotFoundError:
    print("⚠️ Usando dados fictícios.")
    df = pd.DataFrame({'is_fraud_pred': [0]*90 + [1]*10, 'total_in': [1000]*100, 'role': ['Honest']*90+['Mule']*10})
    total_transacoes = 100000

# Função para adicionar borda preta em qualquer eixo
def add_border(ax):
    # Desenha um retângulo preto na borda do eixo
    rect = Rectangle((0, 0), 1, 1, transform=ax.transAxes, linewidth=1.5, edgecolor='black', facecolor='none')
    ax.add_patch(rect)

# ==============================================================================
# IMAGEM 1: DASHBOARD (KPIs + ROSCAS) - COM CONTORNO E TEXTO BRANCO
# ==============================================================================
fig = plt.figure(figsize=(16, 10), dpi=300) 
gs = gridspec.GridSpec(2, 4, height_ratios=[0.25, 0.75]) # Ajustei altura
gs.update(wspace=0.3, hspace=0.3) # Espaço entre os quadros

# KPIs
kpi_contas = len(df)
kpi_fraudes = df['is_fraud_pred'].sum()
kpi_valor = df[df['is_fraud_pred'] == 1]['total_in'].sum()

# Função de Cartão com Borda
def draw_card(ax, t, v, c):
    ax.axis('off')
    # Texto
    ax.text(0.5, 0.6, v, fontsize=22, fontweight='bold', color=c, ha='center', va='center')
    ax.text(0.5, 0.25, t, fontsize=10, color='gray', ha='center', va='center')
    # Borda
    add_border(ax)

# Desenhando Cartões
draw_card(fig.add_subplot(gs[0, 0]), "Total Transações", f"{total_transacoes:,.0f}".replace(',', '.'), "#333")
draw_card(fig.add_subplot(gs[0, 1]), "Contas Únicas", f"{kpi_contas:,.0f}".replace(',', '.'), "#333")
draw_card(fig.add_subplot(gs[0, 2]), "Contas Fraudulentas", f"{kpi_fraudes:,.0f}".replace(',', '.'), "#333")
draw_card(fig.add_subplot(gs[0, 3]), "Valor em Risco", f"R$ {kpi_valor/1e6:.1f}M", "#333")

# --- ROSCA 1: CLASSIFICAÇÃO ---
ax_p1 = fig.add_subplot(gs[1, 0:2])
sizes1 = [kpi_contas - kpi_fraudes, kpi_fraudes]
labels1 = ['Normal', 'Fraude']
wedges, texts, autotexts = ax_p1.pie(sizes1, labels=labels1, colors=PALETTE_PRED, autopct='%1.1f%%', 
                                     explode=(0, 0.1), pctdistance=0.85, startangle=90)

# Estilizando a % (Branco e Negrito)
for autotext in autotexts:
    autotext.set_color('white')
    autotext.set_fontsize(14)
    autotext.set_fontweight('bold')

ax_p1.add_artist(plt.Circle((0,0),0.70,fc='white'))
ax_p1.set_title('Distribuição Total de Classes', fontweight='bold', pad=20)
# Borda em volta do gráfico inteiro (não da rosca, mas da área)
# Para borda em gráfico polar/pizza, precisamos de um truque ou desenhar no retângulo da figura
# Aqui, vou adicionar um retângulo "invisível" em volta para servir de moldura
rect_ax1 = fig.add_subplot(gs[1, 0:2], frameon=False)
rect_ax1.set_xticks([])
rect_ax1.set_yticks([])
add_border(rect_ax1)


# --- ROSCA 2: AUDITORIA ---
ax_p2 = fig.add_subplot(gs[1, 2:4])
fraudes = df[df['is_fraud_pred'] == 1]
if 'role' in fraudes.columns:
    rc = fraudes['role'].value_counts()
    cols = [PALETTE_ROLES.get(r, 'gray') for r in rc.index]
    wedges2, texts2, autotexts2 = ax_p2.pie(rc, labels=rc.index, colors=cols, autopct='%1.1f%%', 
                                            pctdistance=0.85, startangle=140)
    
    # Estilizando a % (Branco e Negrito)
    for autotext in autotexts2:
        autotext.set_color('white')
        autotext.set_fontsize(14)
        autotext.set_fontweight('bold')

    ax_p2.add_artist(plt.Circle((0,0),0.70,fc='white'))
    ax_p2.set_title('Distribuição de Papéis Entre os Fraudulentos', fontweight='bold', pad=20)
    
    # Moldura
    rect_ax2 = fig.add_subplot(gs[1, 2:4], frameon=False)
    rect_ax2.set_xticks([])
    rect_ax2.set_yticks([])
    add_border(rect_ax2)

plt.savefig(f'{save_path}/dashboard_kpi.png', bbox_inches='tight')
plt.close()

# (Os outros gráficos Scatter e Barras continuam iguais, pois já tinham borda ou fundo branco limpo. 
# Se quiser borda neles também, avise!)

print(f"✅ Dashboard atualizado salvo em: {os.path.abspath(save_path)}")
# ==============================================================================
# IMAGEM 2: SCATTER PLOT
# ==============================================================================
plt.figure(figsize=(12, 7), dpi=300)

if 'ratio' in df.columns and 'avg_retention_hours' in df.columns:
    # 1. Filtro Visual
    df_viz = df[(df['ratio'] <= 1.5) & (df['avg_retention_hours'] < 200)]
    
    # 2. Definição Explícita de Cores
    # Garante que 0=Azul Claro e 1=Azul Escuro
    palette_dict = {0: COLOR_NORMAL, 1: COLOR_FRAUD}
    
    # 3. Plotagem (CORREÇÃO: Adicionamos 'ax =')
    # hue_order=[0, 1] garante a ordem para a legenda bater com ['Não', 'Sim']
    ax = sns.scatterplot(
        data=df_viz, 
        x='ratio', 
        y='avg_retention_hours', 
        hue='is_fraud_pred', 
        palette=palette_dict,
        hue_order=[0, 1], 
        alpha=0.6, 
        s=50
    )

    plt.title('Padrão de Comportamento: Dispersão de Classes', fontweight='bold', fontsize=16)
    plt.xlabel('Ratio (Saída/Entrada)', fontsize=12)
    plt.ylabel('Retenção (Horas)', fontsize=12)

    # 4. Ajuste da Legenda
    # Como forçamos a ordem [0, 1] ali em cima, podemos confiar que handles[0] é o Não e handles[1] é o Sim
    try:
        handles, labels = ax.get_legend_handles_labels()
        ax.legend(handles=handles, labels=['Não', 'Sim'], title='É Fraude?', 
                  title_fontsize=12, fontsize=11)
    except:
        pass # Caso o gráfico venha vazio por falta de dados

    # 5. Borda Preta
    add_border(ax)
    plt.savefig(f'{save_path}/scatter_plot.png', bbox_inches='tight')
    plt.close()
else:
    print("Colunas 'ratio' ou 'avg_retention_hours' não encontradas para o scatter plot.")

# ==============================================================================
# IMAGEM 3: BARRAS (CONEXÕES)
# ==============================================================================
if 'in_degree' in df.columns and 'out_degree' in df.columns:
    plt.figure(figsize=(10, 6), dpi=300)
    df_bar = df.groupby('is_fraud_pred')[['in_degree', 'out_degree']].mean().reset_index()
    df_melt = df_bar.melt(id_vars='is_fraud_pred')
    df_melt['variable'] = df_melt['variable'].map({'in_degree': 'In Degree (Recebe)', 'out_degree': 'Out Degree (Envia)'})
    palette_dict = {0: COLOR_NORMAL, 1: COLOR_FRAUD}

    ax = sns.barplot(data=df_melt, x='variable', y='value', hue='is_fraud_pred', palette=palette_dict, hue_order=[0, 1],)
    plt.title('Média de Graus de Entrada e Saída por Classe', fontweight='bold', fontsize=16)
    plt.ylabel('Média de Conexões')
    plt.xlabel('')

    try:
        handles, labels = ax.get_legend_handles_labels()
        ax.legend(handles=handles, labels=['Não', 'Sim'], title='É Fraude?', 
                  title_fontsize=12, fontsize=11)
    except:
        pass

    plt.savefig(f'{save_path}/bar_chart_class.png', bbox_inches='tight')
    plt.close()
else:
    print("Colunas de grau (degree) não encontradas para o gráfico de barras.")


✅ Dashboard atualizado salvo em: c:\Users\usuario\Desktop\Data Science\Projeto AML\assets


In [31]:

plt.rcParams['font.family'] = 'sans-serif'
sns.set_style("white")

# Cores
COLOR_FRAUD = "#0D1B5E"   # Azul Escuro
COLOR_NORMAL = "#0085FF"  # Azul Claro
COLOR_TEXT = "#333333"

# --- 2. DADOS DOS MODELOS ---
dados_modelos = {
    'Modelo': ['XGBoost', 'Random Forest', 'Logistic Reg'],
    'AUC': [0.99, 0.97, 0.98],
    'Recall': [0.92, 0.88, 0.92],
    'Precision': [0.98, 0.97, 0.75]
}
df_metrics = pd.DataFrame(dados_modelos).set_index('Modelo')

# --- 3. CARREGAR PREDIÇÕES DO XGBOOST ---
try:
    df_preds = pd.read_csv('../data/01_raw/resultados_finais_modelagem.csv', index_col=0)
    if 'predicao_classe' in df_preds.columns:
        df_preds = df_preds.rename(columns={'predicao_classe': 'is_fraud_pred'})
    y_real = df_preds['is_fraud_real']
    y_pred = df_preds['is_fraud_pred']
except FileNotFoundError:
    print("Usando dados fictícios.")
    y_real = [0]*90 + [1]*10
    y_pred = [0]*91 + [1]*9

# ==============================================================================
# FUNÇÃO AUXILIAR: BORDA PRETA
# ==============================================================================
def add_border(ax):
    for spine in ax.spines.values():
        spine.set_edgecolor('black')
        spine.set_linewidth(1.5)

# ==============================================================================
# GRÁFICO A: BARRAS COMPARATIVAS
# ==============================================================================
def plot_bars(ax=None):
    # Lógica corrigida: flag para saber se criamos a figura aqui dentro
    is_standalone = ax is None
    if is_standalone: 
        fig, ax = plt.subplots(figsize=(8, 5), dpi=300)
    
    df_melted = df_metrics[['Recall', 'Precision']].reset_index().melt(id_vars='Modelo')
    sns.barplot(data=df_melted, x='Modelo', y='value', hue='variable', 
                palette=[COLOR_NORMAL, COLOR_FRAUD], ax=ax)
    
    ax.set_title('Comparativo de Performance: XGBoost Lidera', fontweight='bold', color=COLOR_TEXT)
    ax.set_ylim(0.5, 1.05)
    ax.set_ylabel('Score (0-1)')
    ax.set_xlabel('')
    ax.legend(title='Métrica', loc='lower right')
    add_border(ax)
    
    if is_standalone: return fig

# Salvar Individual
fig_bars = plot_bars()
if fig_bars:
    fig_bars.savefig(f'{save_path}/bar_chart_models.png', bbox_inches='tight')
    plt.close()

# ==============================================================================
# GRÁFICO B: TABELA (HEATMAP DE TEXTO)
# ==============================================================================
def plot_table(ax=None):
    is_standalone = ax is None
    if is_standalone: 
        fig, ax = plt.subplots(figsize=(6, 3), dpi=300)
    
    ax.axis('off')
    col_labels = ['AUC', 'Recall', 'Precision']
    row_labels = df_metrics.index
    cell_text = [[f"{row.AUC:.2f}", f"{row.Recall:.2f}", f"{row.Precision:.2f}"] for row in df_metrics.itertuples()]
        
    the_table = ax.table(cellText=cell_text, rowLabels=row_labels, colLabels=col_labels, 
                         loc='center', cellLoc='center')
    
    the_table.auto_set_font_size(False)
    the_table.set_fontsize(12)
    the_table.scale(1, 1.8)

    # Colorir vencedor
    COR_DESTAQUE = '#0085FF'

    cells = the_table.get_celld()
    for col_idx, col_name in enumerate(col_labels):
        max_val = df_metrics[col_name].max()
        for row_idx, val in enumerate(df_metrics[col_name]):
            cell = cells[(row_idx+1, col_idx)]
            if val == max_val: 
                # Se for o vencedor: Fundo Azul + Texto Branco (para contraste)
                cell.set_facecolor(COR_DESTAQUE)
                cell.get_text().set_color('white') 
                cell.get_text().set_weight('bold')
            else:
                cell.set_facecolor('white')
                cell.get_text().set_color('black')

    ax.set_title('Resumo Técnico das Métricas', fontweight='bold', y=0.9)
    # Borda na imagem da tabela
    ax.add_patch(plt.Rectangle((0, 0), 1, 1, transform=ax.transAxes, linewidth=2, edgecolor='black', facecolor='none'))
    
    if is_standalone: return fig

# Salvar Individual
fig_table = plot_table()
if fig_table:
    fig_table.savefig(f'{save_path}/model_metrics.png', bbox_inches='tight')
    plt.close()

# ==============================================================================
# GRÁFICO C: MATRIZ DE CONFUSÃO
# ==============================================================================
def plot_confusion(ax=None):
    is_standalone = ax is None
    if is_standalone: 
        fig, ax = plt.subplots(figsize=(5, 5), dpi=300)
    
    cm = confusion_matrix(y_real, y_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False, 
                annot_kws={"size": 16, "weight": "bold"}, ax=ax)
    
    ax.set_title('Matriz de Confusão (XGBoost)', fontweight='bold', color=COLOR_TEXT)
    ax.set_ylabel('Real (Gabarito)')
    ax.set_xlabel('Predito (Modelo)')
    ax.set_xticklabels(['Normal', 'Fraude'])
    ax.set_yticklabels(['Normal', 'Fraude'])
    add_border(ax)
    
    if is_standalone: return fig

# Salvar Individual
fig_cm = plot_confusion()
if fig_cm:
    fig_cm.savefig(f'{save_path}/fraud_matrix.png', bbox_inches='tight')
    plt.close()


print(f"✅ Sucesso! Imagens geradas em: {os.path.abspath(save_path)}")

✅ Sucesso! Imagens geradas em: c:\Users\usuario\Desktop\Data Science\Projeto AML\assets
